In [104]:
class Length:
    __metric = {"mm" : 0.001, "cm" : 0.01, "m" : 1, "km" : 1000,
                "in" : 0.0254, "ft" : 0.3048, "yd" : 0.9144,
                "mi" : 1609.344 }
    
    def __init__(self, val, unit = 'm'):
        self.val = val
        self.unit = unit
    def to_meter(self):
        return Length. __metric[self.unit] * self.val
    def __add__(self, other):
        adden = other * Length.__metric[self.unit] if type(other) in (int, float) else other.to_meter()
        #total = self.to_meter() + other.to_meter()
        total = self.to_meter() + adden
        return Length(total / Length.__metric[self.unit], self.unit)
    def __radd__(self, other):
        return self.__add__(other)
    def __iadd__(self, other):
        adden = other * Length.__metric[self.unit] if type(other) in (int, float) else other.to_meter()
        #total = self.to_meter() + other.to_meter()
        total = self.to_meter() + adden
        self.val = total / Length.__metric[self.unit]
        return self
    def __str__(self):
        return f'{self.val} {self.unit}'
    def __repr__(self):
        return f'Length({self.val}, "{self.unit}")'
    

In [105]:
x = Length(4)

In [106]:
print(x)

4 m


In [107]:
rep = repr(x)
rep

'Length(4, "m")'

In [108]:
eval(rep)

Length(4, "m")

In [109]:
y = Length(300, 'cm')


In [110]:
print(x + y) #adding class Length

7.0 m


In [111]:
x + 2

Length(6.0, "m")

In [112]:
z = Length(3, 'yd')

In [113]:
z + 10 #adding a constant

Length(13.0, "yd")

In [114]:
z += 2

In [103]:
z

Length(5.0, "yd")

In [115]:
5 + z

Length(10.0, "yd")

# Call  

object as function

In [116]:
class Person:
    def __init__(self, name):
        self.name = name
    def __call__(self, height, weight):
        return weight / pow(height, 2)


In [118]:
Trung = Person('Trung')
Trung(1.75, 71)

23.183673469387756

# Excercise: Implement poly1d in numpy

In [345]:
import numpy as np
import re

In [606]:
class poly1d:
    def __init__(self, coefs):
        if np.isscalar(coefs): 
            coefs = [coefs]        #turn a scalar into an array
        n = len(coefs)
        ix = np.nonzero(coefs)[0]  #indices of non zero coefficients
        self.order = n - ix.min() - 1 if len(ix) else 0
        self.c = np.array(coefs[n - self.order - 1:], ndmin = 1, dtype = np.float64)
    def deriv(self, m = 1):
        if self.order == 0:
            return poly1d(0)
        #coefficents of the derivative
        #the order of the derivative polynomial is subtracted by 1 from the order of current polynomial
        deriv_c = np.multiply(self.c,np.arange(self.order, -1, -1))[:-1]
        #the derivative polynomial
        res = poly1d(deriv_c)
        return res if m == 1 else res.deriv(m - 1)
    def integ(self, m = 1, k = 0):
        
        #turn scalar to a tuple
        if np.isscalar(k):
            k = (k,)
        #the cofficent of the integral of the polynomial, increse order by 1, add constant k[-1]
        integ_c = np.append(self.c / np.arange(self.order + 1, 0, -1), k[0])
        #the integral polynomial
        res = poly1d(integ_c)
        if m == 1:
            return res
        
        return res.integ(m - 1, k[1:])
    def __add__(self, poly):
        c = poly1d.operate(self.c, poly.c, np.add)
        return poly1d(c)
    def __iadd__(self, poly):
        return self.__add__(poly)
    def __radd__(self, poly):
        return self.__add__(poly)
    def __sub__(self, poly):
        c = poly1d.operate(self.c, poly.c, np.subtract)
        return poly1d(c)
    def __isub__(self, poly):
        return self.__sub__(poly)
    def __rsub__(self, poly):
        return poly.__sub__(self)
    def __repr__(self):
        return f'poly1d({self.c.tolist()})'
    def __str__(self):
        n = self.order - np.arange(len(self.c))
        #strip + at the beginning
        res = ''.join(poly1d.symbol(self.c, n)).lstrip('+') 
        #if the order or the polinomial > 0, replace the constant 0 with '' (e.g : x + 0 --> x)
        if self.order > 0:
            res = res.rstrip('+0').rstrip('+0.0')
        #adding space at the beginning and end of '+' or '-'
        return re.sub(r'(\+|\-)', r' \1 ', res)
    @staticmethod
    @np.vectorize
    def symbol(c, n):
        c_str = str(c) if c < 0 else f'+{c}' #turn -1 --> 1, 1 --> +1
        if n == 0: return c_str
        return f'{c_str}*x^{n}' if c else '' 
    @staticmethod
    #performing an operation function to two 1 dimensional ndarray
    #resize and filling with 0 if necessary
    # e.g: operation([1,1], [1,1,1,1]) 
    #we have to resize [0,1] to [0,0,1,1] to match the shape of [1,1,1,1]
    #then call operation([0,0,1,1], [0,1,2,3])
    def operate(c1, c2, operation):
        #get the order of 2 array of cofficents
        order1, order2 = len(c1), len(c2)
        order = np.maximum(order1, order2) #the order of the result polynomial
        p1 = np.zeros(order)
        p2 = np.zeros(order)
        p1[order - order1:] = c1
        p2[order - order2:] = c2
        return operation(p1, p2)
    
    def __call__(self, x):
        return np.power(x, np.arange(self.order, -1, -1)) @ self.c
        
        

In [529]:
func1 = poly1d([0,2,1,1])
func1.order

2

In [530]:
print(func1)

2.0*x^2 + 1.0*x^1 + 1


In [531]:
func2 = poly1d([2,5])
func2.order

1

In [532]:
print(func2)

2.0*x^1 + 5


In [533]:
#evaluating at x = 3
func = poly1d([1,2,1])
func(3)

16.0

In [534]:
#adding 2 polynomials
func3 = poly1d([1,2,1]) + poly1d([5,3,-2,4])
print(func3)

5.0*x^3 + 4.0*x^2 + 5


In [535]:
#iadd
func5 = poly1d([1,0,1])
func5 += poly1d([1,1,1])
print(func5)

2.0*x^2 + 1.0*x^1 + 2


In [554]:
#subtracting 2 polynomials
func4 = poly1d([3,2,1]) - poly1d([4,3,5,0])
print(func4)

 - 4.0*x^3 - 3.0*x^1 + 1


In [555]:
#isub
func6 = poly1d([3,2,1])
func6 -= poly1d([4,3,5,0])
print(func6)

 - 4.0*x^3 - 3.0*x^1 + 1


In [567]:
#The derivative polynomial
p = poly1d([4,3,2,1])
deriv1 = p.deriv() #first derivative
deriv2 = p.deriv(2) #second derivative
print('Polynomial:', p)
print('First derivative:',deriv1)
print('Second derivative:', deriv2)

Polynomial: 4.0*x^3 + 3.0*x^2 + 2.0*x^1 + 1
First derivative: 12.0*x^2 + 6.0*x^1 + 2
Second derivative: 24.0*x^1 + 6


In [607]:
#The integral polynomial
p = poly1d(1)
integ2 = p.integ(m = 2, k = (2, 3))
print(integ2)

0.5*x^2 + 2.0*x^1 + 3
